In [1]:
####Package Imports####
from skimage.metrics import structural_similarity
import imutils
import cv2
import os
from collections import Counter
from pprint import pprint as pp
import numpy as np
import anvil.media
import numpy as np
import csv

####File Locations####

location_compare = 'C:/Users/geral/Documents/SeniorProject/Compare'
location_predict = 'C:/Users/geral/Documents/SeniorProject/Predict'

####Connecting to Anvil#####

import anvil.server
anvil.server.connect("TATIRUIDUHZPTWDWQSVZR3SN-GAK2P3JOWGIWVQHG")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [2]:
@anvil.server.callable

####Main Comparison Function####

def compare_image(file1, file2):
    
        with anvil.media.TempFile(file1) as FN1:
            image1 = cv2.imread(FN1)
        with anvil.media.TempFile(file2) as FN2: 
            image2 = cv2.imread(FN2)
            
            
        dim1 = image1.shape
        dim2 = image2.shape
        chk_dim = np.array_equal(dim1,dim2)
        
        ####Normalization####
        
        if (chk_dim is True):
            
            scale_percent = 35

            width1 = int(image1.shape[1] * scale_percent / 100)
            height1 = int(image1.shape[0] * scale_percent / 100)

            width2 = int(image2.shape[1] * scale_percent / 100)
            height2 = int(image2.shape[0] * scale_percent / 100)

            dsize1 = (width1, height1)
            dsize2 = (width2, height2)

            R1 = cv2.resize(image1,dsize1)
            R2 = cv2.resize(image2,dsize2)
        
        
            ####Convert to Grayscale Image####
    
            grayR1 = cv2.cvtColor(R1, cv2.COLOR_BGR2GRAY)
            grayR2 = cv2.cvtColor(R2, cv2.COLOR_BGR2GRAY)

            ####Find Similarity Index####
       

            (score, diff) = structural_similarity(grayR1, grayR2, full=True)

            diff = (diff * 255).astype("uint8")##convert from floating point to unsigned int for openCV
        
            thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
        
            
            for c in cnts:
            
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(R1, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.rectangle(R2, (x, y), (x + w, y + h), (0, 0, 255), 2)
                coords1 = np.asarray(c)
                coords2 = np.squeeze(coords1, axis = 1)
                

            
            np.savetxt((os.path.join(location_compare  , 'Coordinates_Compare.csv')),coords2, fmt = '%.2f', delimiter =',')
            
            ####Save Images to Local File Path####
       
            cv2.imwrite(os.path.join(location_compare  , 'Floorplan1.png'), R1)
            cv2.imwrite(os.path.join(location_compare  , 'Floorplan2.png'), R2)
            cv2.imwrite(os.path.join(location_compare  , 'Difference.png'), diff)
            cv2.imwrite(os.path.join(location_compare  , 'Threshold.png'), thresh)

            return(float(score))
        
        else:
            
            return('error')
            

In [3]:
@anvil.server.callable

def return_image():
    
    f1 = anvil.media.from_file((os.path.join(location_compare , 'Floorplan1.png')), "img/png")
    f2 = anvil.media.from_file((os.path.join(location_compare, 'Floorplan2.png')), "img/png")
    diff = anvil.media.from_file((os.path.join(location_compare , 'Difference.png')), "img/png")
        
    return(f1, f2, diff)



In [4]:
@anvil.server.callable

def return_coordinates_csv():
    coordscsv = anvil.media.from_file((os.path.join(location_compare , 'Coordinates_Compare.csv')), "text/csv")
    return coordscsv

In [5]:
@anvil.server.callable

def return_images_download():
    f1_d = anvil.media.from_file((os.path.join(location_compare , 'Floorplan1.png')), "img/png")
    f2_d = anvil.media.from_file((os.path.join(location_compare, 'Floorplan2.png')), "img/png")
    diff_d = anvil.media.from_file((os.path.join(location_compare, 'Difference.png')), "img/png")
    
    return (f1_d,f2_d,diff_d)

In [12]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

####

####

@anvil.server.callable

def predict_image(imageA):
    """Predict."""
    # [START automl_vision_object_detection_predict]
    from google.cloud import automl
    
    with anvil.media.TempFile(imageA) as FN1:

        imgA = cv2.imread(FN1)
        height = imgA.shape[0]
        width = imgA.shape[1]
    
     
    
    file_path = os.path.join(location_predict , 'Floorplanpredict.png')
    
    cv2.imwrite(file_path, imgA)
    
    ####Load Google API Credentials####
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:/Users/geral/Documents/SeniorProj/FloorPlans-b0febaf27c5c.json'

    ####Variables to Choose Model####
    project_id = "floorplan-274512";
    location = "us-central1";
    model_id = "IOD1541394355862896640";
    prediction_client = automl.PredictionServiceClient()

    ####Obtain Full Path of Model####
    
    model_full_id = prediction_client.model_path(project_id, location , model_id)
    
    #Open Image Through File Path and Take Bytes
    
    with open(file_path, "rb") as content_file:
        content = content_file.read()

    image = automl.types.Image(image_bytes=content)
    payload = automl.types.ExamplePayload(image=image)

    # score_threshold is used to filter the result
    # https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
    
    ###Access JSON Payload#######
    
    ###Google API Function to Return Objects with Score >= 0.8
    params = {"score_threshold": "0.8"} 
    response = prediction_client.predict(model_full_id, payload, params)
   
   
            
        
    ####Prepare Lists for Appending Information####
    object_count = 0
    object_list = []
    score_list = []
    percentage_list = []
    x = []
    y = []
    
    ###Iterate through JSON Payload#######
    
    for result in response.payload:
        
        bounding_box = result.image_object_detection.bounding_box
        object_list.append(result.display_name)
        score_list.append(result.image_object_detection.score)
        percentage_list.append("{:.2%}".format(result.image_object_detection.score)) 
        object_count = object_count + 1
        
         ####Iterate Through Bounding Box in Payload####
        for vertex in bounding_box.normalized_vertices:
            
        ####Multiply Normalized Vertex By Original Dimensions to Get Orignal Vertices######
        
            org_x = int(vertex.x*width) 
            org_y = int(vertex.y*height)
            x.append(org_x)
            y.append(org_y)
             
    ###Create a Tuple of the Original Vertices####
    coords_list = list(zip(x,y))
    np.savetxt((os.path.join(location_predict  , 'Coordinates_Predictions.csv')),coords_list, fmt = '%.2f', delimiter =',')
    
   
    object_count_list = dict(Counter(object_list))
        
    
    
    with open(os.path.join(location_predict  , 'Objects_Count.txt'), "w") as text_file:
        text_file.write("Elements in the Floorplan\n") 
        for k, v in object_count_list.items():
            text_file.write("{} : {}\n".format(k, v))
        text_file.write("Total Elements: {}\n".format(object_count)) 
    
   
    i = 0
    
    for p, coords in enumerate(coords_list):
        
        if p%2 == 0:
            c1 = coords 
            label = object_list[i]
            score = percentage_list[i]
            out = label +" "+ score
            i=i+1
            (a,b) = c1
            cv2.putText(imgA,out,(a,b-10), cv2.FONT_HERSHEY_COMPLEX,1.5,(0,0,255),2)
        else:
            c2 = coords
            cv2.rectangle(imgA, c1, c2, (0, 0, 255), 2)
            
     
    
  
    cv2.imwrite(os.path.join(location_predict, 'predicted.png'), imgA)
      
                
             

    # [END automl_vision_object_detection_predict]
 



    
    

In [13]:
@anvil.server.callable

def return_predict_image():
    preds = anvil.media.from_file((os.path.join(location_predict , 'predicted.png')), "img/png")
    return preds
    
        


In [14]:
@anvil.server.callable

def return_predict_image_download():
    preds = anvil.media.from_file((os.path.join(location_predict , 'predicted.png')), "img/png")
    return preds
    

In [15]:
@anvil.server.callable

def return_predict_csv():
    preds = anvil.media.from_file((os.path.join(location_predict , 'Coordinates_Predictions.csv')), "txt/csv")
    return preds
    
        

In [16]:
@anvil.server.callable

def return_predict_results():
    results = anvil.media.from_file((os.path.join(location_predict , 'Objects_Count.txt')), "txt/plain")
    return results

In [17]:
@anvil.server.callable

def return_predict_results_download():
    results = anvil.media.from_file((os.path.join(location_predict , 'Objects_Count.txt')), "txt/plain")
    return results